In [2]:
import numpy as np
import tensorflow as tf
import time

In [3]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('/home/higgamo/ten/fashionmnist')



Extracting /home/higgamo/ten/fashionmnist/train-images-idx3-ubyte.gz
Extracting /home/higgamo/ten/fashionmnist/train-labels-idx1-ubyte.gz
Extracting /home/higgamo/ten/fashionmnist/t10k-images-idx3-ubyte.gz
Extracting /home/higgamo/ten/fashionmnist/t10k-labels-idx1-ubyte.gz


In [5]:
x_train = mnist.train.images
x_test = mnist.test.images

In [4]:
x_train.shape

(55000, 784)

In [13]:

tf.logging.set_verbosity(tf.logging.ERROR)


def cnn_model_fn(features, labels, mode):
  """Model function for CNN."""
  # Input Layer
  # Reshape X to 4-D tensor: [batch_size, width, height, channels]
  # MNIST images are 28x28 pixels, and have one color channel
  input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])

  # Convolutional Layer #1
  # Computes 32 features using a 5x5 filter with ReLU activation.
  # Padding is added to preserve width and height.
  # Input Tensor Shape: [batch_size, 28, 28, 1]
  # Output Tensor Shape: [batch_size, 28, 28, 32]
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=16,
      kernel_size=[3, 3],
      padding="same",
      activation=tf.nn.relu)
  
    
  # Pooling Layer #1
  # First max pooling layer with a 2x2 filter and stride of 2
  # Input Tensor Shape: [batch_size, 28, 28, 32]
  # Output Tensor Shape: [batch_size, 30, 30, 16]
  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2,2], strides=2)
  
  # Convolutional Layer #2
  # Computes 64 features using a 5x5 filter.
  # Padding is added to preserve width and height.
  # Input Tensor Shape: [batch_size, 14, 14, 32]
  # Output Tensor Shape: [batch_size, 14, 14, 64]
  conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=32,
      kernel_size=[2, 2],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #2
  # Second max pooling layer with a 2x2 filter and stride of 2
  # Input Tensor Shape: [batch_size, 14, 14, 64]
  # Output Tensor Shape: [batch_size, 7, 7, 64]
  pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

  # Flatten tensor into a batch of vectors
  # Input Tensor Shape: [batch_size, 7, 7, 64]
  # Output Tensor Shape: [batch_size, 7 * 7 * 64]
  pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 32])

  # Dense Layer
  # Densely connected layer with 1024 neurons
  # Input Tensor Shape: [batch_size, 7 * 7 * 64]
  # Output Tensor Shape: [batch_size, 1024]
  dense = tf.layers.dense(inputs=pool2_flat, units=512, activation=tf.nn.relu)

  # Add dropout operation; 0.7 probability that element will be kept
  dropout = tf.layers.dropout(
      inputs=dense, rate=0.3, training=mode == tf.estimator.ModeKeys.TRAIN)

  # Logits layer
  # Input Tensor Shape: [batch_size, 1024]
  # Output Tensor Shape: [batch_size, 10]
  logits = tf.layers.dense(inputs=dropout, units=10)

  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }
  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)


def main(unused_argv):
  start_time = time.time()
  
  # Load training and eval data
  mnist = input_data.read_data_sets('/home/higgamo/ten/fashionmnist') 
  train_data = mnist.train.images  # Returns np.array
  train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
  eval_data = mnist.test.images  # Returns np.array
  eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)

  # Create the Estimator
  mnist_classifier = tf.estimator.Estimator(
      model_fn=cnn_model_fn, model_dir="/home/higgamo/ten/model4")

  # Set up logging for predictions
  # Log the values in the "Softmax" tensor with label "probabilities"
  tensors_to_log = {"probabilities": "softmax_tensor"}
  logging_hook = tf.train.LoggingTensorHook(
      tensors=tensors_to_log, every_n_iter=50)
  
  accuracy_list = []
  loss_list = []
  
  # Train the model
  for interation in range(100):
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": train_data},
      y=train_labels,
      batch_size=100,
      num_epochs=None,
      shuffle=True)
    mnist_classifier.train(
      input_fn=train_input_fn,
      steps=20,
      hooks=[logging_hook])

    # Evaluate the model and print results
    eval_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": eval_data},
      y=eval_labels,
      num_epochs=1,
      shuffle=False)
    eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
    print(eval_results)
    accuracy_list.append(eval_results['accuracy'])
    loss_list.append(eval_results['loss'])
    if (interation % 1000 == 0):
        print("--- %s seconds ---" % (time.time() - start_time))
  np.save("/home/higgamo/ten/datasettest/test(1e-2)a",accuracy_list)
  np.save("/home/higgamo/ten/datasettest/test(1e-2)l",loss_list)
  



if __name__ == "__main__":
  tf.app.run()

Extracting /home/higgamo/ten/fashionmnist/train-images-idx3-ubyte.gz
Extracting /home/higgamo/ten/fashionmnist/train-labels-idx1-ubyte.gz
Extracting /home/higgamo/ten/fashionmnist/t10k-images-idx3-ubyte.gz
Extracting /home/higgamo/ten/fashionmnist/t10k-labels-idx1-ubyte.gz
{'accuracy': 0.979, 'loss': 0.10118194, 'global_step': 4280}
--- 3.724255323410034 seconds ---
{'accuracy': 0.9849, 'loss': 0.06214495, 'global_step': 4300}
{'accuracy': 0.9844, 'loss': 0.06439974, 'global_step': 4320}
{'accuracy': 0.9823, 'loss': 0.07153899, 'global_step': 4340}
{'accuracy': 0.9844, 'loss': 0.0682659, 'global_step': 4360}
{'accuracy': 0.985, 'loss': 0.058915373, 'global_step': 4380}
{'accuracy': 0.9853, 'loss': 0.05653932, 'global_step': 4400}
{'accuracy': 0.9833, 'loss': 0.064706996, 'global_step': 4420}
{'accuracy': 0.984, 'loss': 0.060557257, 'global_step': 4440}
{'accuracy': 0.9791, 'loss': 0.08855306, 'global_step': 4460}
{'accuracy': 0.9855, 'loss': 0.054017376, 'global_step': 4480}
{'accuracy

SystemExit: 

/home/higgamo/anaconda3/envs/ravi_project/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2918: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [12]:
test = np.load("/home/higgamo/ten/datasettest/test(1e-2)a.npy")
test.shape

(10,)